In [ ]:
 #!/bin/bash
 # Phenotype counts
 plink --bfile dent1 --freq --missing --make-just-fam --out QC_summary
 
 # Sex Check
 plink --bfile dent1 --check-sex --out QC0
 
 # Run Script 1 in R
 #Remove Problem
 plink --bfile dent1 --remove QC0.problem --make-bed --out data.afterQC0
 
 # Missingness (Individuals)
 plink --bfile data.afterQC0 --missing --out QC1
 
 # Run script 2
 # Remove bad ids
 plink --bfile data.afterQC0 --remove QC1.badids --make-bed --out data.afterQC1
 
 # Missingness (SNPs)
 plink --bfile data.afterQC1 --missing --out QC2
 
 # Run Script 3
 #Remove bad SNPs
 plink --bfile data.afterQC1 --exclude QC2.badsnp --make-bed --out
 data.afterQC2
 
 # LD Pruning
 plink --bfile data.afterQC2 --indep-pairwise 50 5 0.2 --out data.r02
sh extract_SNPs_LRLDregions.sh
 plink --bfile data.afterQC2 --extract data.r02.prune.in --exclude
 longrangeLDregionSNPs.txt --make-bed  --out data.afterQC2.r02

 # Relatedness
 plink --bfile data.afterQC2.r02 --genome --min 0.2 --out data.IBD
 
 # PCA Round 1
 plink --bfile data.afterQC2.r02 --pca header tabs --out data.pca
 
 # Run Script 4
 #PCA Round 2
 plink --bfile data.afterQC2.r02 --maf 0.05 --keep QC3.pcaids --pca header tabs--out data.pca2
 
 # Run Script 5
 # Heterozygosity
 plink --bfile data.afterQC2 --het --keep QC3.pcaids  --out QC4
 
 # Script 6
 #HWE
 plink --bfile data.afterQC2 --keep QC3.pcaids --hwe midp 10e-6 --make-bed --
 out data.clean
 # GWAS
 plink --autosome --bfile data.clean --logistic --hide-covar --covar
 data.pca2.eigenvec --parameters 1-10 --out data.gwas